In [1]:
import numpy as np
import pandas as pd
# import adata
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import os
import datetime
import argparse
import torch
from utils import *
from torch.utils.data import DataLoader
import episcanpy.api as epi

torch.autograd.set_detect_anomaly(True)

## Loading data

For more information check this [link](https://openproblems.bio/neurips_docs/about_tasks/task3_joint_embedding/)

In [2]:
# data
# !ls ~/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2
# ! cd ~/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2
# ! ls /user/leuven/331/vsc33180/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2
! dir ..\

 Volume in drive C has no label.
 Volume Serial Number is 00C5-C2CA

 Directory of C:\Users\Nacho\Desktop\Nacho s stuff\RKMs

07/12/2021  15:40    <DIR>          .
07/12/2021  15:40    <DIR>          ..
07/12/2021  15:31             5.063 environment.yml
07/12/2021  15:39             3.474 environment_nobuilds.yml
07/12/2021  15:20    <DIR>          raw_data
14/12/2021  15:45    <DIR>          RKM_Francesco
               2 File(s)          8.537 bytes
               4 Dir(s)  215.307.620.352 bytes free


In [3]:
os.listdir("../raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2/")

['openproblems_bmmc_multiome_phase2.censor_dataset.output_mod1.h5ad',
 'openproblems_bmmc_multiome_phase2.censor_dataset.output_mod2.h5ad',
 'openproblems_bmmc_multiome_phase2.censor_dataset.output_solution.h5ad']

In [4]:
data_folder_path = "../raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2/"
files = os.listdir(data_folder_path)
# print(data_folder_path + files[0])
data1 = sc.read_h5ad(data_folder_path + files[0])
data2 = sc.read(data_folder_path + files[1])
# data3 = sc.read(data_folder_path + files[2])

Only considering the two last: ['.output_mod2', '.h5ad'].
Only considering the two last: ['.output_mod2', '.h5ad'].


In [5]:
print(data1)
print(data2)

AnnData object with n_obs × n_vars = 42492 × 13431
    obs: 'batch', 'size_factors'
    var: 'gene_ids', 'feature_types'
    uns: 'dataset_id', 'organism'
    layers: 'counts'
AnnData object with n_obs × n_vars = 42492 × 116490
    obs: 'batch'
    var: 'feature_types'
    uns: 'dataset_id', 'gene_activity_var_names', 'organism'
    obsm: 'gene_activity'
    layers: 'counts'


In [6]:
# print(data1.var["feature_types"])
rna = pd.DataFrame.sparse.from_spmatrix(data1.X) # 42492 x 13431
rna.columns = data1.var.index #data1.var["gene_ids"] -> ENSEMBL IDs

sc.pp.highly_variable_genes(data1, n_top_genes=5000) # Find HGVs
rna = rna.transpose().loc[data1.var["highly_variable"] == True].transpose()
rna.head() # 42492 x 5000

rna.to_csv("RNA data preprocessed.csv")

In [7]:
epi.pp.select_var_feature(data2, nb_features=10000, show=False) # Inplace select 10000 variable regions
atac = pd.DataFrame.sparse.from_spmatrix(data2.X)
atac.columns = data2.var.index
atac.head() # 42492 x 10002

atac.to_csv("ATAC data preprocessed.csv")

## Setting up the necessary arguments and functions

In [8]:
# %tb
# # Model Settings =================================================================================================
# parser = argparse.ArgumentParser(description='Gen-RKM Model')

# parser.add_argument('--N', type=int, default=1000, help='Total # of samples')
# parser.add_argument('--mb_size', type=int, default=300, help='Mini-batch size. See utils.py')
# parser.add_argument('--h_dim', type=int, default=2, help='Dim of latent vector') # shared latent space
# parser.add_argument('--capacity', type=int, default=32, help='Capacity of network. See utils.py') # important in CNNs
# parser.add_argument('--x_fdim', type=int, default=128, help='Input x_fdim. See utils.py') # feature map dimensionality
# parser.add_argument('--y_fdim', type=int, default=20, help='Input y_fdim. See utils.py') # feature map dimensionality
# parser.add_argument('--c_accu', type=float, default=100, help='Input weight on recons_error')

# # Training Settings =============================
# parser.add_argument('--lr', type=float, default=1e-4, help='Input learning rate for optimizer')
# parser.add_argument('--max_epochs', type=int, default=100, help='Input max_epoch for cut-off')
# # parser.add_argument('--device', type=str, default='cuda', help='Device type: cuda or cpu')
# parser.add_argument('--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu', help='Device type: cuda or cpu')
# parser.add_argument('--workers', type=int, default=0, help='# of workers for dataloader')
# parser.add_argument('--shuffle', type=bool, default=True, help='shuffle dataset: true or false')

# opt = parser.parse_args()
args = {
    "N": rna.shape[0],
#     "mb_size": 300,
    "h_dim": 32,
    "capacity": 32,
    "x_fdim": 128,
    "y_fdim": 256,
    "c_accu": 100,
    "lr": 1e-5, #1e-6
    "max_epochs": 100,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "workers": 0,
    "shuffle": True,
    "batch_size": 300,
}
opt = args
print(opt["device"])

cuda


In [9]:
def kPCA(X, Y):
#     print(X)
#     print(Y)
    a = torch.mm(X, torch.t(X)) + torch.mm(Y, torch.t(Y))
    nh1 = a.size(0)
    oneN = torch.div(torch.ones(nh1, nh1), nh1).float().to(opt["device"])
#     a = a - torch.mm(oneN, a) - torch.mm(a, oneN) + torch.mm(torch.mm(oneN, a), oneN)  # centering
    print(f"Norm of X {torch.norm(X)}, Norm of Y {torch.norm(Y)}, Norm of a {torch.norm(a)}")
#     print(a, end="\r")

#     Stabilize a(?)
    a = a + 100
    h, s, _ = torch.svd(a, some=False)
    return h[:, :opt["h_dim"]], s

In [10]:
# Energy function
def rkm_loss(output1, X, output2, Y):
    h, s = kPCA(output1, output2)
    U = torch.mm(torch.t(output1), h)
    V = torch.mm(torch.t(output2), h)

    x_tilde = net3(torch.mm(h, torch.t(U))) # NET3
#     y_tilde = net4(torch.mm(h, torch.t(V))) # NET4
    y_tilde = net3(torch.mm(h, torch.t(V))) # NET4

    # Costs
    f1 = torch.trace(torch.mm(torch.mm(output1, U), torch.t(h))) + torch.trace(
        torch.mm(torch.mm(output2, V), torch.t(h)))
    f2 = 0.5 * torch.trace(torch.mm(h, torch.mm(torch.diag(s[:opt["h_dim"]]), torch.t(h))))
    f3 = 0.5 * ((torch.trace(torch.mm(torch.t(U), U))) + (torch.trace(torch.mm(torch.t(V), V))))
    recon_loss = torch.nn.MSELoss()
    f4 = recon_loss(x_tilde, X) + recon_loss(y_tilde, Y)  # reconstruction loss

    loss = - f1 + f3 + f2 + 0.5 * (- f1 + f3 + f2) ** 2 + opt["c_accu"] * f4  # stabilized loss
    return loss

In [11]:
class create_dirs:
    """ Creates directories for Checkpoints and saving trained models """

    def __init__(self, ct):
        self.ct = ct
        self.dircp = 'checkpoint.pth_{}.tar'.format(self.ct)
        self.dirout = 'Mul_trained_RKM_{}.tar'.format(self.ct)

    def create(self):
        if not os.path.exists('cp/'):
            os.makedirs('cp/')

        if not os.path.exists('out/'):
            os.makedirs('out/')

    def save_checkpoint(self, state, is_best):
        if is_best:
            torch.save(state, 'cp/{}'.format(self.dircp))

In [12]:
# https://discuss.pytorch.org/t/dataloader-access-two-items-at-the-same-time/22664
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, dataset1, dataset2):
#         self.dataset1 = dataset1 # datasets should be sorted!
#         self.dataset2 = dataset2

        self.dataset1 = torch.tensor(dataset1.values) # datasets should be sorted!
        self.dataset2 = torch.tensor(dataset2.values)

    def __getitem__(self, index):
#         x1 = self.dataset1.loc[index].to_numpy() #loc, que no iloc (puede que iloc funcione)
#         x2 = self.dataset2.loc[index].to_numpy()
        
        x1 = self.dataset1[index].float()
        x2 = self.dataset2[index].float()

        return x1, x2, index

    def __len__(self):
        return len(self.dataset1) # assuming both datasets have same length

    
train_loader = DataLoader(
             ConcatDataset(rna, atac),
             batch_size=args["batch_size"], shuffle=args["shuffle"],
             num_workers=args["workers"]
            )

## Setting up the NNs

In [13]:
class Net1(nn.Module): # Phi 1 - RNA to feature map
    def __init__(self, input_dim, hidden_dim_list, args):
        super(Net1, self).__init__()
        
        self.transformations = nn.ModuleList()
        self.transformations.append(nn.Linear(in_features=input_dim, out_features=hidden_dim_list[0]))
        
        for i in range(len(hidden_dim_list)):
            if i == len(hidden_dim_list)-1:
                dim = hidden_dim_list[-1]
                self.transformations.append(nn.Linear(in_features=dim, out_features=args["x_fdim"]))
                
            else:
                in_dim, out_dim = hidden_dim_list[i], hidden_dim_list[i+1]
                self.transformations.append(nn.Linear(in_features=in_dim, out_features=out_dim))
        
    def forward(self, x):
        for i, transform in enumerate(self.transformations):
            if i == len(self.transformations)-1:
                x = transform(x)
                # No leaky relu at the end
                
            else:
                x = transform(x)
                x = F.leaky_relu(x, negative_slope=0.2)

        return x
    
    
class Net3(nn.Module):
    def __init__(self, input_dim, hidden_dim_list, args): #Psi 1 - Feature map to RNA (input dim = 5000)
        super(Net3, self).__init__()
        
        hidden_dim_list.reverse() # Reverse the list to be the inverse of Net 1
        self.reverse_transformations = nn.ModuleList()
        self.reverse_transformations.append(nn.Linear(in_features=args["x_fdim"], out_features=hidden_dim_list[0]))
        
        for i in range(len(hidden_dim_list)):
            if i == len(hidden_dim_list)-1:
                dim = hidden_dim_list[-1]
                self.reverse_transformations.append(nn.Linear(in_features=dim, out_features=input_dim))
                
            else:
                in_dim, out_dim = hidden_dim_list[i], hidden_dim_list[i+1]
                self.reverse_transformations.append(nn.Linear(in_features=in_dim, out_features=out_dim))
        
    def forward(self, x):
        for i, transform in enumerate(self.reverse_transformations):
            if i == len(self.reverse_transformations)-1:
                x = transform(x)
                x = F.relu(x) # RNA preprocessed has a lower bound of 0
                
            else:
                x = transform(x)
                x = F.leaky_relu(x, negative_slope=0.2)
                
        return x

In [14]:
class Net2(nn.Module): # Phi 2 - ATAC to feature map
    def __init__(self, input_dim, hidden_dim_list, args): 
        super(Net2, self).__init__()
#         c = capacity
        self.transformations = nn.ModuleList()
        self.transformations.append(nn.Linear(in_features=input_dim, out_features=hidden_dim_list[0]))
        
        for i in range(len(hidden_dim_list)):
            if i == len(hidden_dim_list)-1:
                dim = hidden_dim_list[-1]
                self.transformations.append(nn.Linear(in_features=dim, out_features=args["y_fdim"]))
                
            else:
                in_dim, out_dim = hidden_dim_list[i], hidden_dim_list[i+1]
                self.transformations.append(nn.Linear(in_features=in_dim, out_features=out_dim))
        
    def forward(self, x):
        for i, transform in enumerate(self.transformations):
            if i == len(self.transformations)-1:
                x = transform(x)
                # No leaky relu at the end
                
            else:
                x = transform(x)
                x = F.leaky_relu(x, negative_slope=0.2)
                
        return x


class Net4(nn.Module):
    def __init__(self, input_dim, hidden_dim_list, args): #Psi 2 - Feature map to ATAC (input dim = 116490)
        super(Net4, self).__init__()
    
        hidden_dim_list.reverse() # Reverse the list to be the inverse of Net 2
        self.reverse_transformations = nn.ModuleList()
        self.reverse_transformations.append(nn.Linear(in_features=args["y_fdim"], out_features=hidden_dim_list[0]))
        
        for i in range(len(hidden_dim_list)):
            if i == len(hidden_dim_list)-1:
                dim = hidden_dim_list[-1]
                self.reverse_transformations.append(nn.Linear(in_features=dim, out_features=input_dim))
                
            else:
                in_dim, out_dim = hidden_dim_list[i], hidden_dim_list[i+1]
                self.reverse_transformations.append(nn.Linear(in_features=in_dim, out_features=out_dim))
        
    def forward(self, x):
        for i, transform in enumerate(self.reverse_transformations):
            if i == len(self.reverse_transformations)-1:
                x = transform(x)
                x = torch.sigmoid(x) # ATAC is has binary values
                
            else:
                x = transform(x)
                x = F.leaky_relu(x, negative_slope=0.2) 
                
        return x   

In [15]:
# args = {"x_fdim":2}
# hid = [1000, 500, 300, 200, 50, 10]
# inp = 10000
# n2 = Net2(inp, hid, args)
# n4 = Net4(inp, hid, args)
# print(n2.transformations)
# print(n4.reverse_transformations)

In [16]:
def final_compute(data, net1, net2, kPCA, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    """ Function to compute embeddings of full dataset. """
    # Data is a dictionary with keys RNA and ATAC
#     args["shuffle"] = False
#     xt, _, _ = get_mnist_dataloader(args=args)  # loading data without shuffle
    xt = data["RNA"]
    yt = data["ATAC"]
    xtr = net1(xt.to(args.device))
    ytr = net2(yt.to(args.device)) # This was targets at first, but we don't have labels

    h, s = kPCA(xtr, ytr)
    return torch.mm(torch.t(xtr), h), torch.mm(torch.t(ytr), h), h, s

## Initialize nets and parameters

In [17]:
# net1 = Net1(input_dim=5000, hidden_dim=1000, args=args).float().to(opt["device"]) # input_dim, hidden_dim, args
# net2 = Net2(input_dim=116490, hidden_dim_list=[10000, 2000], args=args).float().to(opt["device"]) # input_dim, hidden_dim_list, args
# net3 = Net3(input_dim=5000, hidden_dim=1000, args=args).float().to(opt["device"]) # input_dim, hidden_dim, args
# net4 = Net4(input_dim=116490, hidden_dim_list=[10000, 2000], args=args).float().to(opt["device"]) # input_dim, hidden_dim_list, args
# net4 = Net4(input_dim=116490, hidden_dim_list=[10000, 2000], args=args).double().to(opt["device"])

hidden_list_rna = [3000, 1000, 500, 250]
hidden_list_atac = [5000, 2500, 1000, 500, 300]
net1 = Net1(input_dim=5000, hidden_dim_list=hidden_list_rna, args=args).float().to(opt["device"]) # input_dim, hidden_dim, args
net2 = Net2(input_dim=10002, hidden_dim_list=hidden_list_atac, args=args).float().to(opt["device"]) # input_dim, hidden_dim_list, args
net3 = Net3(input_dim=5000, hidden_dim_list=hidden_list_rna, args=args).float().to(opt["device"]) # input_dim, hidden_dim, args
net4 = Net4(input_dim=10002, hidden_dim_list=hidden_list_atac, args=args).float().to(opt["device"])

params = list(net1.parameters()) + list(net3.parameters()) + list(net2.parameters()) + list(net4.parameters())
optimizer = torch.optim.Adam(params, lr=opt["lr"], weight_decay=0)

## Train

In [18]:
ct = time.strftime("%Y%m%d-%H%M")
dirs = create_dirs(ct=ct)
dirs.create()

In [19]:
# float(sum([torch.max(p) for p in net1.parameters()]))
#int(sum([torch.sum(torch.isnan(p)) for p in net1.parameters()]))
# float(max([torch.max(p.grad) for p in net1.parameters()]))

In [20]:
l_cost = np.inf  # Costs from where checkpoints will be saved
t = 1
cost = np.inf  # Initialize cost
start = datetime.datetime.now()
while cost > 0.2 and t <= opt["max_epochs"]:  # run epochs until convergence
    avg_loss = 0
    for i, (datax, datay, _) in enumerate(train_loader):
        if i < math.ceil(opt["N"] / opt["batch_size"]):
            print(f'On step {i}', end="\r")
#             print(f'On step {i}')
            datax, datay = datax.float().to(opt["device"]), datay.float().to(opt["device"])
            output1 = net1(datax.float())
            print(f"Max of net1 {float(sum([torch.max(p) for p in net1.parameters()]))}, #nan of net1 {int(sum([torch.sum(torch.isnan(p)) for p in net1.parameters()]))}")
            output2 = net2(datay.float())
            loss = rkm_loss(output1, datax, output1, datax).float()

            optimizer.zero_grad()
            loss.backward()
            print(f"Loss is {float(loss)}, max of grad is {float(max([torch.max(p.grad) for p in net1.parameters()]))}")
#             nn.utils.clip_grad_norm_(parameters=params, max_norm=1)
            optimizer.step()
            avg_loss += loss.detach().cpu().numpy()
        
        else:
            break
    cost = avg_loss

    # Remember lowest cost and save checkpoint
    is_best = cost < l_cost
    l_cost = min(cost, l_cost)

    dirs.save_checkpoint({
        'epochs': t + 1,
        'net1_state_dict': net1.state_dict(),
        'net3_state_dict': net3.state_dict(),
        'net2_state_dict': net2.state_dict(),
        'net4_state_dict': net4.state_dict(),
        'l_cost': l_cost,
        'optimizer': optimizer.state_dict()}, is_best)
    print(f"\n\nEpoch: {t}, Cost: {cost}\n")
    t += 1
print('Finished Training in: {}. Lowest cost: {}'.format(str(datetime.datetime.now() - start), l_cost))

Max of net1 0.34276241064071655, #nan of net1 0
Norm of X 7.919551849365234, Norm of Y 7.919551849365234, Norm of a 123.98271942138672
Loss is 112473336.0, max of grad is 41.57763671875
Max of net1 0.3428015112876892, #nan of net1 0
Norm of X 7.932462692260742, Norm of Y 7.932462692260742, Norm of a 124.35760498046875
Loss is 112522144.0, max of grad is 17.041624069213867
Max of net1 0.3428414463996887, #nan of net1 0
Norm of X 7.9553351402282715, Norm of Y 7.9553351402282715, Norm of a 124.9958724975586
Loss is 112489896.0, max of grad is 25.08282470703125
Max of net1 0.3428851366043091, #nan of net1 0
Norm of X 7.972089767456055, Norm of Y 7.972089767456055, Norm of a 125.54137420654297
Loss is 112508720.0, max of grad is 43.64646911621094
Max of net1 0.342912495136261, #nan of net1 0
Norm of X 7.978144645690918, Norm of Y 7.978144645690918, Norm of a 125.76471710205078
Loss is 112504904.0, max of grad is 22.154468536376953
Max of net1 0.3429397642612457, #nan of net1 0
Norm of X 7.9

Max of net1 0.3442678451538086, #nan of net1 0
Norm of X 8.243221282958984, Norm of Y 8.243221282958984, Norm of a 134.0102081298828
Loss is 112512816.0, max of grad is 20.335773468017578
Max of net1 0.3442971706390381, #nan of net1 0
Norm of X 8.238688468933105, Norm of Y 8.238688468933105, Norm of a 133.80104064941406
Loss is 112512520.0, max of grad is 20.251237869262695
Max of net1 0.34432312846183777, #nan of net1 0
Norm of X 8.251620292663574, Norm of Y 8.251620292663574, Norm of a 134.2218017578125
Loss is 112518584.0, max of grad is 24.36795425415039
Max of net1 0.344348669052124, #nan of net1 0
Norm of X 8.25137710571289, Norm of Y 8.25137710571289, Norm of a 134.28631591796875
Loss is 112520472.0, max of grad is 94.82470703125
Max of net1 0.34438225626945496, #nan of net1 0
Norm of X 8.26294994354248, Norm of Y 8.26294994354248, Norm of a 134.57237243652344
Loss is 112497400.0, max of grad is 60.612152099609375
Max of net1 0.34441709518432617, #nan of net1 0
Norm of X 8.28496

Max of net1 0.34556281566619873, #nan of net1 0
Norm of X 8.657148361206055, Norm of Y 8.657148361206055, Norm of a 147.41542053222656
Loss is 112511448.0, max of grad is 25.22409439086914
Max of net1 0.3455968499183655, #nan of net1 0
Norm of X 8.672333717346191, Norm of Y 8.672333717346191, Norm of a 147.7610626220703
Loss is 112513632.0, max of grad is 40.54978561401367
Max of net1 0.3456309735774994, #nan of net1 0
Norm of X 8.677132606506348, Norm of Y 8.677132606506348, Norm of a 147.94972229003906
Loss is 112496232.0, max of grad is 21.111328125
Max of net1 0.34566599130630493, #nan of net1 0
Norm of X 8.678248405456543, Norm of Y 8.678248405456543, Norm of a 148.00653076171875
Loss is 112503968.0, max of grad is 22.782575607299805
Max of net1 0.3456987142562866, #nan of net1 0
Norm of X 8.67471981048584, Norm of Y 8.67471981048584, Norm of a 147.87049865722656
Loss is 112512912.0, max of grad is 27.567609786987305
Max of net1 0.34572723507881165, #nan of net1 0
Norm of X 8.6711

Max of net1 0.3465256690979004, #nan of net1 0
Norm of X 8.956082344055176, Norm of Y 8.956082344055176, Norm of a 157.70753479003906
Loss is 112489872.0, max of grad is 28.8709716796875
Max of net1 0.34654152393341064, #nan of net1 0
Norm of X 8.988251686096191, Norm of Y 8.988251686096191, Norm of a 158.78367614746094
Loss is 112516272.0, max of grad is 22.113332748413086
Max of net1 0.34655481576919556, #nan of net1 0
Norm of X 9.001988410949707, Norm of Y 9.001988410949707, Norm of a 159.23291015625
Loss is 112495232.0, max of grad is 28.234954833984375
Max of net1 0.34657177329063416, #nan of net1 0
Norm of X 8.999495506286621, Norm of Y 8.999495506286621, Norm of a 159.0738067626953
Loss is 112511536.0, max of grad is 14.630008697509766
Max of net1 0.34658774733543396, #nan of net1 0
Norm of X 9.01222038269043, Norm of Y 9.01222038269043, Norm of a 159.60623168945312
Loss is 112515536.0, max of grad is 19.360624313354492
Max of net1 0.3466027081012726, #nan of net1 0
Norm of X 8.

Max of net1 0.3471716642379761, #nan of net1 0
Norm of X 9.217162132263184, Norm of Y 9.217162132263184, Norm of a 166.91062927246094
Loss is 112508224.0, max of grad is 21.571666717529297
Max of net1 0.34718751907348633, #nan of net1 0
Norm of X 9.204341888427734, Norm of Y 9.204341888427734, Norm of a 166.40052795410156
Loss is 112519640.0, max of grad is 29.904613494873047
Max of net1 0.3472006320953369, #nan of net1 0
Norm of X 9.241433143615723, Norm of Y 9.241433143615723, Norm of a 167.5551300048828
Loss is 112488304.0, max of grad is 30.58917236328125
Max of net1 0.3472178876399994, #nan of net1 0
Norm of X 9.217690467834473, Norm of Y 9.217690467834473, Norm of a 166.8814239501953
Loss is 112495736.0, max of grad is 23.253934860229492
Max of net1 0.3472391366958618, #nan of net1 0
Norm of X 9.228569030761719, Norm of Y 9.228569030761719, Norm of a 167.28526306152344
Loss is 112518624.0, max of grad is 15.178428649902344
Max of net1 0.3472576439380646, #nan of net1 0
Norm of X 

...

Norm of X 1202.7603759765625, Norm of Y 1202.7603759765625, Norm of a 2832954.5
Loss is 109579224.0, max of grad is 132955.859375
Max of net1 0.5823768973350525, #nan of net1 0
Norm of X 1202.287353515625, Norm of Y 1202.287353515625, Norm of a 2833112.5
Loss is 109307368.0, max of grad is 139635.53125
Max of net1 0.5823647975921631, #nan of net1 0
Norm of X 1199.8221435546875, Norm of Y 1199.8221435546875, Norm of a 2818778.25
Loss is 109251936.0, max of grad is 154697.28125
Max of net1 0.5823562145233154, #nan of net1 0
Norm of X 1200.1353759765625, Norm of Y 1200.1353759765625, Norm of a 2822258.75
Loss is 109392408.0, max of grad is 159023.078125
Max of net1 0.5823439359664917, #nan of net1 0
Norm of X 1206.781005859375, Norm of Y 1206.781005859375, Norm of a 2853425.25
Loss is 109623640.0, max of grad is 146140.9375
Max of net1 0.5823341012001038, #nan of net1 0
Norm of X 1204.484375, Norm of Y 1204.484375, Norm of a 2839941.0
Loss is 109006320.0, max of grad is 135536.359375
Max 

Loss is 109703240.0, max of grad is 112146.9375
Max of net1 0.5818589925765991, #nan of net1 0
Norm of X 1263.196044921875, Norm of Y 1263.196044921875, Norm of a 3127389.0
Loss is 109532968.0, max of grad is 81583.9375
Max of net1 0.5818504095077515, #nan of net1 0
Norm of X 1260.61328125, Norm of Y 1260.61328125, Norm of a 3115635.5
Loss is 109788432.0, max of grad is 131734.015625
Max of net1 0.5818419456481934, #nan of net1 0
Norm of X 1260.4306640625, Norm of Y 1260.4306640625, Norm of a 3120778.5
Loss is 109579224.0, max of grad is 116259.6953125
Max of net1 0.5818307399749756, #nan of net1 0
Norm of X 1256.16943359375, Norm of Y 1256.16943359375, Norm of a 3093088.25
Loss is 108891920.0, max of grad is 162895.09375
Max of net1 0.5818204879760742, #nan of net1 0
Norm of X 1252.9923095703125, Norm of Y 1252.9923095703125, Norm of a 3082368.5
Loss is 109035856.0, max of grad is 115571.1171875
Max of net1 0.5818110704421997, #nan of net1 0
Norm of X 1259.78271484375, Norm of Y 1259.

Max of net1 0.5811917781829834, #nan of net1 0
Norm of X 1372.998779296875, Norm of Y 1372.998779296875, Norm of a 3700379.5
Loss is 109179880.0, max of grad is 173380.828125
Max of net1 0.5812029242515564, #nan of net1 0
Norm of X 1375.3304443359375, Norm of Y 1375.3304443359375, Norm of a 3717726.0
Loss is 109117080.0, max of grad is 142024.0625
Max of net1 0.5812163352966309, #nan of net1 0
Norm of X 1356.3460693359375, Norm of Y 1356.3460693359375, Norm of a 3613113.5
Loss is 108807080.0, max of grad is 176981.953125
Max of net1 0.5812312364578247, #nan of net1 0
Norm of X 1376.081298828125, Norm of Y 1376.081298828125, Norm of a 3715058.5
Loss is 109434928.0, max of grad is 113250.5
Max of net1 0.5812443494796753, #nan of net1 0
Norm of X 1364.4832763671875, Norm of Y 1364.4832763671875, Norm of a 3657684.25
Loss is 109251936.0, max of grad is 321698.1875
Max of net1 0.581253170967102, #nan of net1 0
Norm of X 1351.973876953125, Norm of Y 1351.973876953125, Norm of a 3589927.75
Lo

Max of net1 0.5814482569694519, #nan of net1 0
Norm of X 1408.9705810546875, Norm of Y 1408.9705810546875, Norm of a 3895674.25
Loss is 108613544.0, max of grad is 234879.25
Max of net1 0.5814502239227295, #nan of net1 0
Norm of X 1390.412353515625, Norm of Y 1390.412353515625, Norm of a 3788783.5
Loss is 108587760.0, max of grad is 182259.171875
Max of net1 0.5814458131790161, #nan of net1 0
Norm of X 1415.925048828125, Norm of Y 1415.925048828125, Norm of a 3936144.75
Loss is 109827328.0, max of grad is 234604.34375
Max of net1 0.5814451575279236, #nan of net1 0
Norm of X 1406.4483642578125, Norm of Y 1406.4483642578125, Norm of a 3883596.0
Loss is 108784952.0, max of grad is 227148.203125
Max of net1 0.581450343132019, #nan of net1 0
Norm of X 1398.085693359375, Norm of Y 1398.085693359375, Norm of a 3834888.25
Loss is 110928672.0, max of grad is 2242411.0
Max of net1 0.5815619826316833, #nan of net1 0
Norm of X 1417.19921875, Norm of Y 1417.19921875, Norm of a 3938922.0
Loss is 108

RuntimeError: Function 'SvdHelperBackward0' returned nan values in its 0th output.

In [ ]:

15909857544.0
15910955228.0
15910548268.0
15910719520.0
15910903424.0
15911022996.0